In [3]:
# coding: utf-8
from urllib import request, parse
import ssl
import json
import time

# 配置变量
config = {
    # 用户ID
    'userId':'',  # TODO, 你的用户id，PC上在币乎页面，按F12，network页面找到对应的请求，可以找到
    # 登录token
    'accessToken':'', # TODO, 你的token，登录后币乎服务器返回，获取方法与userId相同。
	# 粉丝条件，粉丝少于多少个时，取消关注
    'fansCondition':100 # 可以根据你的实际情况设置阈值条件
}

def datetime_str():  
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(time.time())
    dt = time.strftime(format,value)  
    return dt

# 打印日志信息
def print_info(info):
    log = "[%s]--%s" % (datetime_str(), info)
    print(log)

# 取消关注某人    
def unFollow(subjectUserId, userName):
    print_info("取消关注【%s】..." % (userName))
    url = r'https://be02.bihu.com/bihube-pc/api/content/unFollow'
    headers = {
        'User-Agent': r'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
        'Content-Type': r'application/x-www-form-urlencoded;charset=utf-8',
        'Referer': r'https://bihu.com/people/' + config['userId'] + r'/index=2',
        'Connection': 'keep-alive',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9'
    }
    
    data = {
        'userId': config['userId'],
        'accessToken': config['accessToken'],
        'subjectUserId': subjectUserId
    }
    context = ssl._create_unverified_context()
    data = parse.urlencode(data).encode('utf-8')
    req = request.Request(url, headers=headers, data=data)

    rsp = ""
    try:
        rsp = request.urlopen(req, context = context).read()
        rsp = rsp.decode('utf-8')
    except urllib2.URLError as e:
        print_info(str(e.code) + '\r\n' + e.reason);

    print_info("取消关注【%s】返回：%s" % (userName, rsp))

    return rsp

# 获取关注列表，pageNum为页序    
def getUserFollowList(pageNum):
    print_info("查询第【" + str(pageNum) + "】页关注列表" )
    url = r'https://be02.bihu.com/bihube-pc/api/content/show/getUserFollowList'
    headers = {
        'User-Agent': r'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
        'Content-Type': r'application/x-www-form-urlencoded;charset=utf-8',
        'Referer': r'https://bihu.com/people/' + config['userId'] + r'/index=2',
        'Connection': 'keep-alive',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9'
    }
    
    data = {
        'queryUserId': config['userId'],
        'userId': config['userId'],
        'accessToken': config['accessToken'],
        'pageNum': pageNum
    }
    
    context = ssl._create_unverified_context()
    data = parse.urlencode(data).encode('utf-8')
    req = request.Request(url, headers=headers, data=data)

    rsp = ""
    try:
        rsp = request.urlopen(req, context = context).read()
        rsp = rsp.decode('utf-8')
    except urllib2.URLError as e:
        print_info(str(e.code) + '\r\n' + e.reason);

    return rsp

def parse_follow_list(body):
    data_str = json.loads(body);
    res = data_str['res'];

    # 请求是否成功
    is_succ = False;
    pages = 0;
    log_content_format = '【%s】的粉丝数：%d，头像是:%s'
    if res == 1 and data_str["data"]["size"] > 0:
        pages = data_str["data"]["pages"]
        # 遍历关注对象
        for item in data_str["data"]["list"]:
            if item["userName"].startswith("币友_") \
            or item["userIcon"] == "img/bihu_user_default_icon.png"\
            or item["fans"] < config['fansCondition']:
                log_content = log_content_format % (item['userName'], item["fans"], item["userIcon"])
                print_info(log_content)
                # 昵称以币友_开始，userIcon为默认，且粉丝数小于配置的，取消关注
                unFollow(item["userId"],item['userName'])
    
    if res == 1:
        is_succ  = True;

    return is_succ, pages

def Run():
    # 第一次获取，为了拿到页数，好做循环
    pageIndex = 0
    pages = 1
    is_suc = True
 
    while is_suc and pageIndex < pages:
        pageIndex += 1
        time.sleep(1)
        body = getUserFollowList(pageIndex)
        is_suc, pages = parse_follow_list(body)
        
		# 请求太频繁时，进入下面等待，一分钟检测一次
        while (not is_suc):
            print_info(body)
            print_info("60秒后请稍后重试")
            time.sleep(60)
            body = getUserFollowList(pageIndex)
            is_suc, pages = parse_follow_list(body)
            
Run()
print_info("程序已退出")

[2018-04-16 00:33:19]--查询第【1】页关注列表
[2018-04-16 00:33:19]--【币圈故事会】的粉丝数：57，头像是:img/790ee554a3a8e6e8d569e3d34fd5b7b3.jpeg
[2018-04-16 00:33:19]--取消关注【币圈故事会】...
[2018-04-16 00:33:19]--取消关注【币圈故事会】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:33:19]--【币友_48656】的粉丝数：45，头像是:img/bihu_user_default_icon.png
[2018-04-16 00:33:19]--取消关注【币友_48656】...
[2018-04-16 00:33:19]--取消关注【币友_48656】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:33:19]--【stonecat】的粉丝数：56，头像是:img/5cf4b37e6d76f2bcfd4fbe9e4e8020bd.jpeg
[2018-04-16 00:33:19]--取消关注【stonecat】...
[2018-04-16 00:33:20]--取消关注【stonecat】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:33:20]--【币友_63383】的粉丝数：80，头像是:img/bihu_user_default_icon.png
[2018-04-16 00:33:20]--取消关注【币友_63383】...
[2018-04-16 00:33:20]--取消关注【币友_63383】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:33:20]--【币友_63443】的粉丝数：88，头像是:img/bihu_user_default_icon.png
[2018-04-16 00:33:20]--取消关注【币友_63443】...
[2018-04-16 00:33:20]--取消关注【币友_63443】返回：{"d

[2018-04-16 00:33:31]--取消关注【飛天貓】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:33:31]--【火星party】的粉丝数：42，头像是:img/24284edf645db31319b7783cbf7c5de1.jpeg
[2018-04-16 00:33:31]--取消关注【火星party】...
[2018-04-16 00:33:31]--取消关注【火星party】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:33:31]--【币友_225282】的粉丝数：84，头像是:img/fa6533cc39d3712babd61724240953d4.jpeg
[2018-04-16 00:33:31]--取消关注【币友_225282】...
[2018-04-16 00:33:32]--取消关注【币友_225282】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:33:32]--【po塘主】的粉丝数：99，头像是:img/4e1b726cd633bc7c9c7fb17c68363d83.jpeg
[2018-04-16 00:33:32]--取消关注【po塘主】...
[2018-04-16 00:33:32]--取消关注【po塘主】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:33:32]--【币友_226120】的粉丝数：24，头像是:img/bihu_user_default_icon.png
[2018-04-16 00:33:32]--取消关注【币友_226120】...
[2018-04-16 00:33:32]--取消关注【币友_226120】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:33:32]--【道格】的粉丝数：57，头像是:img/c821f713fcdd2fb3c4740e9daf6bf98b.jpg
[2018-04-16 00:33:32]--取消关注【

[2018-04-16 00:33:43]--取消关注【疏星】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:33:43]--【币友_213428】的粉丝数：25，头像是:img/bihu_user_default_icon.png
[2018-04-16 00:33:43]--取消关注【币友_213428】...
[2018-04-16 00:33:43]--取消关注【币友_213428】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:33:43]--【武静_厦大人】的粉丝数：43，头像是:img/bihu_user_default_icon.png
[2018-04-16 00:33:43]--取消关注【武静_厦大人】...
[2018-04-16 00:33:44]--取消关注【武静_厦大人】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:33:44]--【大飞】的粉丝数：381，头像是:img/bihu_user_default_icon.png
[2018-04-16 00:33:44]--取消关注【大飞】...
[2018-04-16 00:33:44]--取消关注【大飞】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:33:44]--【币友_213626】的粉丝数：50，头像是:img/bihu_user_default_icon.png
[2018-04-16 00:33:44]--取消关注【币友_213626】...
[2018-04-16 00:33:44]--取消关注【币友_213626】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:33:44]--【静云】的粉丝数：22，头像是:img/7c057a649e840f2b13f6df4b18d70bab.jpeg
[2018-04-16 00:33:44]--取消关注【静云】...
[2018-04-16 00:33:44]--取消关注【静云】返回：

[2018-04-16 00:33:58]--取消关注【币友_206055】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:33:58]--【郝晓月】的粉丝数：66，头像是:img/34f6243e5e8c744ebdd902d2e43282b9.jpeg
[2018-04-16 00:33:58]--取消关注【郝晓月】...
[2018-04-16 00:33:59]--取消关注【郝晓月】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:33:59]--【firefly】的粉丝数：93，头像是:img/f2e856dbbd864a5681062ddb84885b9c.jpeg
[2018-04-16 00:33:59]--取消关注【firefly】...
[2018-04-16 00:33:59]--取消关注【firefly】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:33:59]--【币友_206460】的粉丝数：74，头像是:img/7b2d9700f68c47329a8b57a33ded167f.jpeg
[2018-04-16 00:33:59]--取消关注【币友_206460】...
[2018-04-16 00:33:59]--取消关注【币友_206460】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:33:59]--【王小诗】的粉丝数：54，头像是:img/f6f48bf1a9459e8d4d9f64ee72a928c7.jpeg
[2018-04-16 00:33:59]--取消关注【王小诗】...
[2018-04-16 00:33:59]--取消关注【王小诗】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:33:59]--【嘟嘟】的粉丝数：41，头像是:img/bihu_user_default_icon.png
[2018-04-16 00:33:59]--取消关注【嘟嘟】...
[2018-0

[2018-04-16 00:34:11]--取消关注【币友_184843】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:34:11]--【小白说事】的粉丝数：59，头像是:img/bihu_user_default_icon.png
[2018-04-16 00:34:11]--取消关注【小白说事】...
[2018-04-16 00:34:12]--取消关注【小白说事】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:34:12]--【币友_185193】的粉丝数：25，头像是:img/e4ed434430c85ea5da0f6463566a122f.jpeg
[2018-04-16 00:34:12]--取消关注【币友_185193】...
[2018-04-16 00:34:12]--取消关注【币友_185193】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:34:12]--【币友_185484】的粉丝数：30，头像是:img/bihu_user_default_icon.png
[2018-04-16 00:34:12]--取消关注【币友_185484】...
[2018-04-16 00:34:12]--取消关注【币友_185484】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:34:12]--【微风细雨】的粉丝数：53，头像是:img/2f0d0389bb4b6ddf84be7ab1649efe77.jpeg
[2018-04-16 00:34:12]--取消关注【微风细雨】...
[2018-04-16 00:34:12]--取消关注【微风细雨】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:34:13]--查询第【10】页关注列表
[2018-04-16 00:34:13]--【币友_152777】的粉丝数：46，头像是:img/bihu_user_default_icon.png
[2018-04-

...

[2018-04-16 00:41:07]--取消关注【币友_28062】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:41:07]--【XIEHU0215】的粉丝数：111，头像是:img/991ea94c2f15fa8164755a58d401d815.jpeg
[2018-04-16 00:41:07]--取消关注【XIEHU0215】...
[2018-04-16 00:41:07]--取消关注【XIEHU0215】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:41:07]--【粉色蚂蚁】的粉丝数：178，头像是:img/87e38e67d319f70c252e07f82b0aab5c.jpeg
[2018-04-16 00:41:07]--取消关注【粉色蚂蚁】...
[2018-04-16 00:41:08]--取消关注【粉色蚂蚁】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:41:09]--查询第【69】页关注列表
[2018-04-16 00:41:09]--【xtdctg】的粉丝数：217，头像是:img/d6c9d62a4b32835830f0d4bc73db84d2.jpeg
[2018-04-16 00:41:09]--取消关注【xtdctg】...
[2018-04-16 00:41:09]--取消关注【xtdctg】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:41:09]--【JohnsonLee】的粉丝数：628，头像是:img/db283541e407ee3419744359f72a519a.jpeg
[2018-04-16 00:41:09]--取消关注【JohnsonLee】...
[2018-04-16 00:41:09]--取消关注【JohnsonLee】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:41:09]--【感悟人生】的粉丝数：299，头像是:img/2e8ae4

[2018-04-16 00:41:20]--取消关注【牛席】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:41:20]--【山鹰】的粉丝数：1979，头像是:img/f38c5f08cea00d56a9a3759b6296e269.jpeg
[2018-04-16 00:41:20]--取消关注【山鹰】...
[2018-04-16 00:41:21]--取消关注【山鹰】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:41:21]--【未来而来33316】的粉丝数：354，头像是:img/ee725041fa561391c0e86221d5147746.jpeg
[2018-04-16 00:41:21]--取消关注【未来而来33316】...
[2018-04-16 00:41:21]--取消关注【未来而来33316】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:41:21]--【小卡】的粉丝数：196，头像是:img/bihu_user_default_icon.png
[2018-04-16 00:41:21]--取消关注【小卡】...
[2018-04-16 00:41:21]--取消关注【小卡】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:41:21]--【币友_33601】的粉丝数：234，头像是:img/bihu_user_default_icon.png
[2018-04-16 00:41:21]--取消关注【币友_33601】...
[2018-04-16 00:41:21]--取消关注【币友_33601】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 00:41:21]--【鑫森淼焱垚】的粉丝数：272，头像是:img/159838ed788b799a9dbe578e0bb057c7.png
[2018-04-16 00:41:21]--取消关注【鑫森淼焱垚】...
[2018-04-16 00:41

[2018-04-16 01:00:34]--取消关注【币友_22602】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 01:00:34]--【群】的粉丝数：338，头像是:img/0034efaa10bb54e9e6d0d8bf942aaa36.jpeg
[2018-04-16 01:00:34]--取消关注【群】...
[2018-04-16 01:00:35]--取消关注【群】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 01:00:35]--【Troy】的粉丝数：235，头像是:img/bcef192de5a3f0fcb9023e14726a46e1.jpeg
[2018-04-16 01:00:35]--取消关注【Troy】...
[2018-04-16 01:00:35]--取消关注【Troy】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 01:00:35]--【何不痛快】的粉丝数：363，头像是:img/a0499b2f4fdd60b4f481f4a8a4ff3320.jpeg
[2018-04-16 01:00:35]--取消关注【何不痛快】...
[2018-04-16 01:00:35]--取消关注【何不痛快】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 01:00:35]--【梁大块】的粉丝数：363，头像是:img/bihu_user_default_icon.png
[2018-04-16 01:00:35]--取消关注【梁大块】...
[2018-04-16 01:00:35]--取消关注【梁大块】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 01:00:35]--【彭文峰】的粉丝数：325，头像是:img/f810c7ce6ea89009ee0a84e66185a601.jpg
[2018-04-16 01:00:35]--取消关注【彭文峰】...
[2018-04-16 01:00:36]--取消关注【彭文峰】

[2018-04-16 01:00:47]--取消关注【华尔资本】返回：{"data":null,"res":0,"resMsg":"币友太热情了, 服务器忙不过来, 请稍后重试！"}
[2018-04-16 01:00:47]--【FaFaFa】的粉丝数：196，头像是:img/7163c3c8cece57013a07613b2a3c9ab8.jpeg
[2018-04-16 01:00:47]--取消关注【FaFaFa】...
[2018-04-16 01:00:47]--取消关注【FaFaFa】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 01:00:47]--【乎啦啦圈】的粉丝数：241，头像是:img/2b6f99bf41c5fc5154abff96d8da4f86.jpeg
[2018-04-16 01:00:47]--取消关注【乎啦啦圈】...
[2018-04-16 01:00:48]--取消关注【乎啦啦圈】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 01:00:48]--【BlockD】的粉丝数：1018，头像是:img/195783184cb3a5bdb79388b62460a878.jpeg
[2018-04-16 01:00:48]--取消关注【BlockD】...
[2018-04-16 01:00:48]--取消关注【BlockD】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 01:00:48]--【冰红茶】的粉丝数：449，头像是:img/ea737be8a4c7f921fe348cc3a2deffd9.jpeg
[2018-04-16 01:00:48]--取消关注【冰红茶】...
[2018-04-16 01:00:48]--取消关注【冰红茶】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 01:00:48]--【阳光心屋】的粉丝数：233，头像是:img/1196df4454e863813031de0be5818025.jpeg
[2018-04-16 01:00:48]

[2018-04-16 01:01:03]--取消关注【悦享】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 01:01:03]--【海外冷门币种分析】的粉丝数：648，头像是:img/f6ecd2b08d70be04e5e559f89f756aaa.jpg
[2018-04-16 01:01:03]--取消关注【海外冷门币种分析】...
[2018-04-16 01:01:03]--取消关注【海外冷门币种分析】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 01:01:04]--查询第【78】页关注列表
[2018-04-16 01:01:04]--【Wolkin】的粉丝数：394，头像是:img/15d198146b54f149aec7e8a19fecf451.jpeg
[2018-04-16 01:01:04]--取消关注【Wolkin】...
[2018-04-16 01:01:04]--取消关注【Wolkin】返回：{"data":null,"res":0,"resMsg":"币友太热情了, 服务器忙不过来, 请稍后重试！"}
[2018-04-16 01:01:04]--【币飞来】的粉丝数：405，头像是:img/938cce695e10f6327ae7e0c2c7b07eb8.jpeg
[2018-04-16 01:01:04]--取消关注【币飞来】...
[2018-04-16 01:01:05]--取消关注【币飞来】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 01:01:05]--【西江月】的粉丝数：1248，头像是:img/b7ee052a11da9ad3bb09ff081b167220.jpeg
[2018-04-16 01:01:05]--取消关注【西江月】...
[2018-04-16 01:01:05]--取消关注【西江月】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-16 01:01:05]--【26号路人甲】的粉丝数：871，头像是:img/c88ab1ad0cacfc80e5f5c